<a href="https://colab.research.google.com/github/Subhashree1202/GCNN_NodeClassification/blob/main/NodeClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import os.path as osp
from PIL import Image as im

import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid,Amazon
from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GCNConv

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import SimpleRNN, LSTM, GRU
from keras.utils import np_utils
from keras import backend as K
from distutils.version import LooseVersion as LV
from keras.utils.vis_utils import model_to_dot
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
from collections import defaultdict
import random
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix

datasets= ['Cora','Citeseer','Pubmed','Computers','Photo']
args_dataset = datasets[0]
args_lr = 0.01
args_epochs = 200

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = osp.join('data', 'Planetoid')
#dataset = Amazon(path, args_dataset, transform=T.NormalizeFeatures())
dataset = Planetoid(path, args_dataset, transform=T.NormalizeFeatures())
data = dataset[0]
data1 = data.to(device)
features = data.x.numpy()
features.reshape((features.shape[0], features.shape[1], 1))

y = data.y.numpy()
y = np_utils.to_categorical(y, dataset.num_classes)

def rnn_model(rows, cols, nb_classes, hidden1, hidden2):
    model = Sequential()

    # Recurrent layers supported: SimpleRNN, LSTM, GRU:
    model.add(SimpleRNN(hidden1,
                        input_shape=(rows, cols),
                        return_sequences=True))
    model.add(SimpleRNN(hidden2))

    model.add(Dense(units=nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    print(model.summary())
    return model

def rnn_feature_model(rows, cols , hidden1, hidden2):
    model = Sequential()

    # Recurrent layers supported: SimpleRNN, LSTM, GRU:
    model.add(SimpleRNN(hidden1,
                        input_shape=(rows, cols),
                        return_sequences=True))
    model.add(SimpleRNN(hidden2))
    return model

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, drops, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, int(hidden_channels), cached=True,
                             normalize=True)
        self.conv2 = GCNConv(int(hidden_channels), out_channels, cached=True,
                             normalize=True)
        self.drops = drops

    def forward(self, x, edge_index, edge_weight=None):
        x = F.dropout(x, p=self.drops, training=self.training)
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=self.drops, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return x

def epsilonGreedy(epsilon, Q, state):
        if np.random.uniform() < epsilon:
            return random.randint(0, 2)
        else:
            best_next_action = np.argmax(Q[state,:])

        return best_next_action

def get_reward_RL_RNN(action, state):
    modelRNN = rnn_model(features.shape[1],1, dataset.num_classes, action, state)
    epochs = 3
    history = modelRNN.fit(features,
                        y,
                        epochs=epochs,
                        batch_size=128,
                        verbose=2)
    return history.history['accuracy'][-1]

def get_reward_RL_GNN(shape1, action, state, ss = False):
    model = GCN(shape1, action, state, dataset.num_classes)
    model = model.to(device)
    optimizer2 = torch.optim.Adam([
        dict(params=model.conv1.parameters(), weight_decay=5e-4),
        dict(params=model.conv2.parameters(), weight_decay=0)
    ], lr=args_lr)

    def train():
        model.train()
        optimizer2.zero_grad()
        out = model(rnn_feature, data1.edge_index, data1.edge_weight)
        loss = F.cross_entropy(out[data1.train_mask], data1.y[data.train_mask])
        loss.backward()
        optimizer2.step()
        return float(loss)


    @torch.no_grad()
    def test():
        model.eval()
        pred = model(rnn_feature, data1.edge_index, data1.edge_weight).argmax(dim=-1)

        actual2 = data1.y[data1.test_mask]
        pred2 = pred[data1.test_mask]

        accs = []
        for mask in [data1.train_mask, data1.val_mask, data1.test_mask]:
            accs.append(int((pred[mask] == data1.y[mask]).sum()) / int(mask.sum()))
        return accs, actual2, pred2


    best_val_acc = final_test_acc = 0
    train_loss = []
    train_acc_list = []
    val_acc_list = []
    test_acc_list = []

    for epoch in range(1, args_epochs + 1):
        loss = train()
        [train_acc, val_acc, tmp_test_acc], actual2, pred2 = test()
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
            actual2_ = actual2
            pred2_ = pred2
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        train_loss.append(loss)
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        test_acc_list.append(test_acc)

    if ss ==True:
        plt.figure()
        plt.plot(train_loss)
        plt.xlabel('epoch')
        plt.ylabel('train loss')

        plt.figure()
        plt.plot(train_acc_list)
        plt.xlabel('epoch')
        plt.ylabel('train accuracy')

        plt.figure()
        plt.plot(val_acc_list)
        plt.xlabel('epoch')
        plt.ylabel('valid accuracy')

        plt.figure()
        plt.plot(test_acc_list)
        plt.xlabel('epoch')
        plt.ylabel('test accuracy')

    return test_acc, actual2_, pred2_

n_episodes=1
##// hidden_numbers = np.array([[800,1000,1200],[1224, 1512, 1624]])
hidden_numbers = np.array([[200,300,400],[250, 350, 450]])
parameters = np.array([[8,16,32],[0.4,0.5,0.6]])
Q_table = np.zeros((3, 3))

def hiddenLayer_opt(nntype):
  lr = 0.1
  gamma = 0.99
  epsilonDecayFactor = 0.99
  epsilon = 1.0
  min_epsilon = 0.1

  for e1 in range(n_episodes):
        print("episode for RL_RNN: ", e1)
        epsilon *= epsilonDecayFactor
        epsilon = max(min_epsilon, epsilon)
        current_state = 0
        step = 0
        for k in range(1):
            step += 1
            action =epsilonGreedy(epsilon, Q_table, current_state)
            if nntype == 'RNN':
              reward = get_reward_RL_RNN(hidden_numbers[0, action],hidden_numbers[1,current_state])
            elif nntype == 'RL':
              reward ,_ ,_= get_reward_RL_GNN(hidden2, parameters[0,action], parameters[1,current_state], False)
            next_state = (action + random.randint(0, 2)) % 3

            best_next_action = np.argmax(Q_table[next_state,:])
            Q_table[current_state][action] = (1-lr) * Q_table[current_state][action] \
                                              + lr*(reward + gamma* Q_table[next_state][best_next_action])

            current_state = next_state

hiddenLayer_opt('RNN')

hidden1 = np.argmax(Q_table, axis=0)[0]
hidden1 = hidden_numbers[0, hidden1]
hidden2 = np.argmax(Q_table, axis=1)[0]
hidden2 = hidden_numbers[1,hidden2]

rnn_feature_m = rnn_feature_model(features.shape[1],1,hidden1, hidden2)
rnn_feature = rnn_feature_m.predict(features)
print('rnn feature shape: ', rnn_feature.shape)
rnn_feature = torch.tensor(rnn_feature)

hiddenLayer_opt('RL')

hidden = np.argmax(Q_table, axis=0)[0]
drop = np.argmax(Q_table, axis=1)[0]

accuracy, actual, pred = get_reward_RL_GNN(hidden2, parameters[0,hidden], parameters[1,drop],True)

confusion_matrix = metrics.confusion_matrix(actual, pred)
print(confusion_matrix)

fig, ax = plot_confusion_matrix(conf_mat=confusion_matrix)
plt.show()